In [1]:
import torch.nn as nn
import torch
from torchvision import transforms, datasets
import numpy as np
import math

## Image Preprocessing

They only mean center so we found the mean pixel value of faces and normalize with that.

In [2]:
data_transform = transforms.Compose([transforms.Grayscale(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.5089547997389491],
                                     std=[1])])
allImages = datasets.ImageFolder(root='./training',transform = data_transform)
label_mapping = torch.FloatTensor([float(clazz) for clazz in allImages.classes])
# label_mappin
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# make it output from [0,1] rather than [1900,2010]
label_mapping_scaled = (label_mapping - label_mapping.min())/(label_mapping.max() - label_mapping.min())

In [4]:
dataloader = torch.utils.data.DataLoader(allImages,batch_size = 128, shuffle=True)

In [5]:
class ResNet(nn.Module):
    
    def __init__(self, n_layers, final_output, bottleneck = False):
        super(ResNet,self).__init__()
        self.conv_params = {'kernel_size': 3, 'padding': 1}
        self.width = 186
        self.height = 171
        self.layer_dict = {
            18: [2,2,2,2],
            34: [3,4,6,3]
        }
        self.layers = {}
        
        in_channels = 1
        out_channels = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 7, stride = 2, padding = 3),
            nn.BatchNorm2d(num_features = out_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1, dilation = 1)
        )
        self.width = self.width / 2
        self.width = math.floor( (self.width - 1) / 2 + 1 )
        self.height = self.height / 2
        self.height = math.floor( (self.height - 1) / 2 + 1)
        print("Height is now:", self.height, "Width is now:", self.width)
        
        
        in_channels = 64
        
        num_repeat = self.layer_dict[n_layers]
        for i in range(2,6):
            self.res_layer = i
            # [ [blocks], transition ]
            self.layers[self.res_layer] = [[], None]
            for j in range(num_repeat[i-2]):
                self.create_block(in_channels, out_channels, j, bottleneck)
                if j == 0:
                    self.add_transition()
                in_channels = out_channels
            out_channels = out_channels * 2
        self.relu = nn.ReLU(inplace=True)
        # global average pooling
        self.global_avg = nn.AvgPool2d(kernel_size = (self.width,self.height), stride = 1)
        # fully connected to final
        self.output = nn.Linear(in_channels,1)
        
        
        
        
    def create_block(self, in_channels, out_channels, block_num, bottleneck):
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, stride = 1, **self.conv_params),
            nn.BatchNorm2d(num_features = out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, stride = 2 if block_num == 0 else 1, **self.conv_params),
            nn.BatchNorm2d(num_features = out_channels)
        )
        self.add_module("conv" + str(self.res_layer) + "_" + str(block_num), block)
        self.layers[self.res_layer][0].append(block)
        print("Added", "conv" + str(self.res_layer) + "_" + str(block_num), "input: " + str(in_channels), "output: " + str(out_channels))
        
        if block_num == 0:
            self.height = math.floor( (self.height - 1) / 2 + 1)
            self.width = math.floor( (self.width - 1) / 2 + 1)
            print("Height is now:", self.height, "Width is now:", self.width)
        
    
    def add_transition(self):
        transition = nn.Sequential(
            nn.AvgPool2d(kernel_size = 3, stride = 2, padding = 1)
        )
        self.add_module("transition"+ str(self.res_layer), transition)
        self.layers[self.res_layer][1] = transition
    
    def forward(self, X):
        # go through conv1
        X = self.conv1(X)
        # go through residuals
        for i in range(2,self.res_layer + 1):
            layers,transition = self.layers[i]
            for j,layer in enumerate(layers):
                pool = X
                if j == 0:          
                    pool = transition(X)
                    # dimension transition
                    if i > 2:
                        padding = (0,0,0,0,pool.shape[1]//2,pool.shape[1]//2,0,0)
                        pool = nn.functional.pad(pool,padding)
                X = layer(X) + pool
                X = self.relu(X)
        X = self.global_avg(X)
        X = X.view(X.shape[0],-1)
        X = self.output(X)
        return X.view(-1)
        

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
resnet34 = ResNet(34,1).to(device)

Height is now: 43 Width is now: 47
Added conv2_0 input: 64 output: 64
Height is now: 22 Width is now: 24
Added conv2_1 input: 64 output: 64
Added conv2_2 input: 64 output: 64
Added conv3_0 input: 64 output: 128
Height is now: 11 Width is now: 12
Added conv3_1 input: 128 output: 128
Added conv3_2 input: 128 output: 128
Added conv3_3 input: 128 output: 128
Added conv4_0 input: 128 output: 256
Height is now: 6 Width is now: 6
Added conv4_1 input: 256 output: 256
Added conv4_2 input: 256 output: 256
Added conv4_3 input: 256 output: 256
Added conv4_4 input: 256 output: 256
Added conv4_5 input: 256 output: 256
Added conv5_0 input: 256 output: 512
Height is now: 3 Width is now: 3
Added conv5_1 input: 512 output: 512
Added conv5_2 input: 512 output: 512


In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
count_parameters(resnet34)

21112705

### Training

In [11]:
optim = torch.optim.Adam(resnet34.parameters(),lr = 0.001, betas = (0.9,0.999))
loss_metric = nn.L1Loss()
n_epochs = 100
iteration = 0
for e in range(n_epochs):
    losses = []
    for batch_input, batch_labels in dataloader:
        if iteration % 100 == 0:
            print(iteration)
        # make sure to zero out gradient
        resnet34.zero_grad()
        
        # move to gpu + get correct labels
        batch_input = batch_input.to(device)
        batch_labels = label_mapping_scaled[batch_labels].to(device)
        
        loss = loss_metric(resnet34(batch_input),batch_labels)
        losses.append(loss.data)
        loss.backward()
        optim.step()
        iteration += 1
#         break
    print("Epoch %d: Training Loss: %0.3f" % (e,np.mean(losses)))

0
100
Epoch 0: Training Loss: 0.162
200
300
Epoch 1: Training Loss: 0.108
400
500
Epoch 2: Training Loss: 0.095
600
700
Epoch 3: Training Loss: 0.087
800
Epoch 4: Training Loss: 0.080
900
1000
Epoch 5: Training Loss: 0.074
1100
1200
Epoch 6: Training Loss: 0.074
1300
1400
Epoch 7: Training Loss: 0.064
1500
1600
Epoch 8: Training Loss: 0.063
1700
Epoch 9: Training Loss: 0.057
1800
1900
Epoch 10: Training Loss: 0.055
2000
2100
Epoch 11: Training Loss: 0.052
2200
2300
Epoch 12: Training Loss: 0.049
2400
2500
Epoch 13: Training Loss: 0.047
2600
Epoch 14: Training Loss: 0.044
2700
2800
Epoch 15: Training Loss: 0.042
2900
3000
Epoch 16: Training Loss: 0.040
3100
3200
Epoch 17: Training Loss: 0.043
3300
3400
Epoch 18: Training Loss: 0.039
3500
Epoch 19: Training Loss: 0.035
3600
3700
Epoch 20: Training Loss: 0.034
3800
3900
Epoch 21: Training Loss: 0.033
4000
4100
Epoch 22: Training Loss: 0.033
4200
Epoch 23: Training Loss: 0.031
4300
4400
Epoch 24: Training Loss: 0.030
4500
4600
Epoch 25: Tr

## Evaluation

In [40]:
data_transform = transforms.Compose([transforms.Grayscale(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.5089547997389491],
                                     std=[1])])
valImages = datasets.ImageFolder(root='./validation',transform = data_transform)
label_mapping_v = torch.FloatTensor([float(clazz) for clazz in valImages.classes])

# label_mapping
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [41]:
label_mapping

tensor([ 1905.,  1906.,  1908.,  1909.,  1910.,  1911.,  1912.,  1913.,
         1914.,  1915.,  1916.,  1919.,  1922.,  1923.,  1924.,  1925.,
         1926.,  1927.,  1928.,  1929.,  1930.,  1931.,  1932.,  1933.,
         1934.,  1935.,  1936.,  1937.,  1938.,  1939.,  1940.,  1941.,
         1942.,  1943.,  1944.,  1945.,  1946.,  1947.,  1948.,  1949.,
         1950.,  1951.,  1952.,  1953.,  1954.,  1955.,  1956.,  1957.,
         1958.,  1959.,  1960.,  1961.,  1962.,  1963.,  1964.,  1965.,
         1966.,  1967.,  1968.,  1969.,  1970.,  1971.,  1972.,  1973.,
         1974.,  1975.,  1976.,  1977.,  1978.,  1979.,  1980.,  1981.,
         1982.,  1983.,  1984.,  1985.,  1986.,  1987.,  1988.,  1989.,
         1990.,  1991.,  1992.,  1993.,  1994.,  1995.,  1996.,  1997.,
         1998.,  1999.,  2000.,  2001.,  2002.,  2003.,  2004.,  2005.,
         2006.,  2007.,  2008.,  2009.,  2010.,  2011.,  2012.,  2013.])

In [42]:
label_mapping_v

tensor([ 1933.,  1935.,  1936.,  1940.,  1944.,  1945.,  1946.,  1947.,
         1949.,  1950.,  1951.,  1952.,  1954.,  1955.,  1959.,  1961.,
         1962.,  1963.,  1965.,  1967.,  1968.,  1970.,  1972.,  1973.,
         1975.,  1976.,  1977.,  1978.,  1979.,  1981.,  1983.,  1984.,
         1990.,  1991.,  1992.,  2000.,  2001.,  2002.,  2005.,  2008.,
         2011.,  2012.])

In [43]:
label_mapping_scaled_v = (label_mapping_v - label_mapping.min())/(label_mapping.max() - label_mapping.min())

In [23]:
label_mapping_scaled_v

tensor([ 0.0000,  0.0253,  0.0380,  0.0886,  0.1392,  0.1519,  0.1646,
         0.1772,  0.2025,  0.2152,  0.2278,  0.2405,  0.2658,  0.2785,
         0.3291,  0.3544,  0.3671,  0.3797,  0.4051,  0.4304,  0.4430,
         0.4684,  0.4937,  0.5063,  0.5316,  0.5443,  0.5570,  0.5696,
         0.5823,  0.6076,  0.6329,  0.6456,  0.7215,  0.7342,  0.7468,
         0.8481,  0.8608,  0.8734,  0.9114,  0.9494,  0.9873,  1.0000])

In [25]:
valDataloader = torch.utils.data.DataLoader(valImages,batch_size = 128,shuffle=True)

In [46]:
# to turn off running averages in batch norm
resnet34.eval()
losses = []
for batch_input,batch_labels in valDataloader:
    batch_input = batch_input.to(device)
    batch_labels = label_mapping_scaled_v[batch_labels].to(device)
    res = resnet34(batch_input)
#     print(res)
#     print(batch_labels)
    loss = loss_metric(res,batch_labels)
#     print(loss.data)
    losses.append(loss.data)
print(np.mean(losses))

0.046557505


In [27]:
print(losses)

[tensor(0.1226, device='cuda:0'), tensor(0.1452, device='cuda:0'), tensor(0.1358, device='cuda:0'), tensor(0.1300, device='cuda:0'), tensor(0.1321, device='cuda:0'), tensor(0.1246, device='cuda:0'), tensor(0.1419, device='cuda:0'), tensor(0.1292, device='cuda:0'), tensor(0.1356, device='cuda:0'), tensor(0.1464, device='cuda:0'), tensor(0.1410, device='cuda:0'), tensor(0.1308, device='cuda:0'), tensor(0.1389, device='cuda:0'), tensor(0.1282, device='cuda:0'), tensor(0.1357, device='cuda:0'), tensor(0.1299, device='cuda:0'), tensor(0.1335, device='cuda:0'), tensor(0.1398, device='cuda:0'), tensor(0.1365, device='cuda:0'), tensor(0.1391, device='cuda:0'), tensor(0.1293, device='cuda:0'), tensor(0.1508, device='cuda:0'), tensor(0.1500, device='cuda:0'), tensor(0.1600, device='cuda:0'), tensor(0.1442, device='cuda:0'), tensor(0.1385, device='cuda:0'), tensor(0.1298, device='cuda:0'), tensor(0.1410, device='cuda:0'), tensor(0.1505, device='cuda:0'), tensor(0.1444, device='cuda:0'), tensor(0.

In [28]:
batch_labels

tensor([ 0.2785,  0.8608,  0.8481,  0.7468,  0.3291,  0.2152,  0.4304,
         0.3797,  0.5823,  0.1392,  0.7468,  0.5443,  0.0380,  0.2785,
         0.4684,  0.4430,  0.6076], device='cuda:0')